# ✂️ Lecture 3: Pruning & Sparsity (Part 1) - Complete Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/efficientml_course/blob/main/03_pruning_sparsity_1/demo.ipynb)

## What You'll Learn
- Weight pruning fundamentals
- Magnitude-based pruning implementation
- Structured vs unstructured pruning
- Sparsity patterns and their hardware implications

In [ ]:
!pip install torch torchvision matplotlib numpy -q
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)
print('Ready for pruning!')

## Part 1: Why Pruning Works - Weight Distribution Analysis

In [ ]:
# Create a simple trained model
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

model = SimpleNet()

# Simulate trained weights (trained models have near-zero weights)
# Real trained networks have many small weights!
with torch.no_grad():
    for param in model.parameters():
        # Many weights become small after training
        mask = torch.rand_like(param) > 0.3
        param.data = param.data * mask.float() * 0.1 + param.data * (~mask).float()

# Visualize weight distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (name, param) in enumerate(model.named_parameters()):
    if 'weight' in name:
        weights = param.data.flatten().numpy()
        axes[idx].hist(weights, bins=100, color='#3b82f6', alpha=0.7, edgecolor='black')
        axes[idx].axvline(x=0, color='red', linestyle='--', linewidth=2)
        axes[idx].set_title(f'{name}\nMean: {np.mean(weights):.4f}, Std: {np.std(weights):.4f}')
        axes[idx].set_xlabel('Weight Value')
        axes[idx].set_ylabel('Count')
        
        # Calculate near-zero weights
        threshold = 0.05
        near_zero = np.sum(np.abs(weights) < threshold) / len(weights) * 100
        axes[idx].text(0.95, 0.95, f'{near_zero:.1f}% near-zero', 
                       transform=axes[idx].transAxes, ha='right', va='top')

plt.suptitle('📊 Weight Distribution in Neural Networks', fontsize=14)
plt.tight_layout()
plt.show()

print('\n💡 Key Insight: Many weights are near-zero and can be removed!')

## Part 2: Implementing Magnitude Pruning from Scratch

In [ ]:
def magnitude_prune(tensor, sparsity):
    """
    Prune weights with smallest magnitude.
    
    Args:
        tensor: Weight tensor to prune
        sparsity: Fraction of weights to remove (0.9 = remove 90%)
    
    Returns:
        Pruned tensor and binary mask
    """
    # Get absolute values
    abs_weights = torch.abs(tensor)
    
    # Find threshold
    num_weights = tensor.numel()
    num_to_prune = int(num_weights * sparsity)
    
    # Get threshold value
    threshold = torch.kthvalue(abs_weights.flatten(), num_to_prune).values
    
    # Create mask
    mask = abs_weights > threshold
    
    # Apply mask
    pruned_tensor = tensor * mask.float()
    
    return pruned_tensor, mask

# Test our implementation
original_weights = model.fc1.weight.data.clone()
pruned_weights, mask = magnitude_prune(original_weights, sparsity=0.9)

print('📊 MAGNITUDE PRUNING RESULTS')
print('=' * 50)
print(f'Original weights: {original_weights.numel():,}')
print(f'Non-zero after pruning: {mask.sum().item():,}')
print(f'Actual sparsity: {100 - mask.sum().item()/mask.numel()*100:.1f}%')

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Original weights heatmap
im0 = axes[0].imshow(original_weights[:50, :50].numpy(), cmap='RdBu', aspect='auto')
axes[0].set_title('Original Weights')
plt.colorbar(im0, ax=axes[0])

# Mask
im1 = axes[1].imshow(mask[:50, :50].numpy(), cmap='binary', aspect='auto')
axes[1].set_title('Pruning Mask (white=keep)')

# Pruned weights
im2 = axes[2].imshow(pruned_weights[:50, :50].numpy(), cmap='RdBu', aspect='auto')
axes[2].set_title('Pruned Weights (90% sparse)')
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()

## Part 3: Using PyTorch's Built-in Pruning

In [ ]:
# Create fresh model
model = SimpleNet()

print('🔧 PYTORCH PRUNING API')
print('=' * 50)

# Before pruning
print('\nBefore pruning:')
print(f'  fc1.weight shape: {model.fc1.weight.shape}')
print(f'  Parameters: {list(model.fc1.named_parameters())}')

# Apply L1 unstructured pruning
prune.l1_unstructured(model.fc1, name='weight', amount=0.9)

print('\nAfter pruning:')
print(f'  Parameters: {list(model.fc1.named_parameters())}')
print(f'  Buffers: {list(model.fc1.named_buffers())}')

# Check sparsity
sparsity = 100 * float(torch.sum(model.fc1.weight == 0)) / float(model.fc1.weight.nelement())
print(f'\n📊 Sparsity achieved: {sparsity:.1f}%')

# Make pruning permanent
prune.remove(model.fc1, 'weight')
print('\nAfter removing pruning reparametrization:')
print(f'  Parameters: {list(model.fc1.named_parameters())}')

## Part 4: Structured vs Unstructured Pruning

In [ ]:
def visualize_pruning_types(weight_matrix):
    """
    Compare different pruning patterns.
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # Original
    axes[0, 0].imshow(weight_matrix, cmap='RdBu', aspect='auto')
    axes[0, 0].set_title('Original Weights', fontsize=12)
    
    # Unstructured (random)
    unstructured_mask = torch.rand_like(weight_matrix) > 0.7
    axes[0, 1].imshow(weight_matrix * unstructured_mask.float(), cmap='RdBu', aspect='auto')
    axes[0, 1].set_title('Unstructured Pruning (70%)\n(Random pattern)', fontsize=12)
    
    # Row pruning (structured - output channels)
    row_mask = torch.ones_like(weight_matrix)
    rows_to_remove = torch.randperm(weight_matrix.shape[0])[:int(weight_matrix.shape[0]*0.7)]
    row_mask[rows_to_remove] = 0
    axes[0, 2].imshow(weight_matrix * row_mask, cmap='RdBu', aspect='auto')
    axes[0, 2].set_title('Row/Filter Pruning (70%)\n(Remove entire filters)', fontsize=12)
    
    # Column pruning (structured - input channels)
    col_mask = torch.ones_like(weight_matrix)
    cols_to_remove = torch.randperm(weight_matrix.shape[1])[:int(weight_matrix.shape[1]*0.7)]
    col_mask[:, cols_to_remove] = 0
    axes[1, 0].imshow(weight_matrix * col_mask, cmap='RdBu', aspect='auto')
    axes[1, 0].set_title('Column Pruning (70%)\n(Remove input channels)', fontsize=12)
    
    # N:M Sparsity (2:4)
    nm_mask = torch.zeros_like(weight_matrix)
    for i in range(weight_matrix.shape[0]):
        for j in range(0, weight_matrix.shape[1], 4):
            if j + 4 <= weight_matrix.shape[1]:
                vals = weight_matrix[i, j:j+4]
                _, top2_idx = vals.abs().topk(2)
                nm_mask[i, j + top2_idx[0]] = 1
                nm_mask[i, j + top2_idx[1]] = 1
    axes[1, 1].imshow(weight_matrix * nm_mask, cmap='RdBu', aspect='auto')
    axes[1, 1].set_title('2:4 Structured Sparsity (50%)\n(Hardware-friendly)', fontsize=12)
    
    # Block sparsity
    block_mask = torch.ones_like(weight_matrix)
    block_size = 8
    for i in range(0, weight_matrix.shape[0], block_size):
        for j in range(0, weight_matrix.shape[1], block_size):
            if torch.rand(1) > 0.3:  # 70% of blocks removed
                block_mask[i:i+block_size, j:j+block_size] = 0
    axes[1, 2].imshow(weight_matrix * block_mask, cmap='RdBu', aspect='auto')
    axes[1, 2].set_title('Block Sparsity (70%)\n(GPU-friendly)', fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Create example weight matrix
test_weights = torch.randn(64, 128)
visualize_pruning_types(test_weights)

print('\n📊 PRUNING TYPES COMPARISON')
print('=' * 60)
print(f'{"Type":<25} {"Speedup":<15} {"Hardware Support":<20}')
print('-' * 60)
print(f'{"Unstructured":<25} {"Minimal":<15} {"Sparse BLAS":<20}')
print(f'{"Row/Column (Structured)":<25} {"High":<15} {"All GPUs":<20}')
print(f'{"2:4 Sparsity":<25} {"2x":<15} {"NVIDIA Ampere+":<20}')
print(f'{"Block Sparsity":<25} {"Good":<15} {"GPU Tensor Cores":<20}')

## Part 5: Pruning with Fine-tuning Pipeline

In [ ]:
# Complete pruning pipeline
from torch.utils.data import DataLoader, TensorDataset

# Create synthetic data
X_train = torch.randn(1000, 784)
y_train = torch.randint(0, 10, (1000,))
X_test = torch.randn(200, 784)
y_test = torch.randint(0, 10, (200,))

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            outputs = model(x)
            _, predicted = outputs.max(1)
            total += y.size(0)
            correct += predicted.eq(y).sum().item()
    return 100 * correct / total

def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    nonzero = sum((p != 0).sum().item() for p in model.parameters())
    return total, nonzero

# Train original model
model = SimpleNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print('📚 TRAINING ORIGINAL MODEL')
for epoch in range(5):
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()

acc_before = evaluate(model, test_loader)
total, nonzero = count_parameters(model)
print(f'Original accuracy: {acc_before:.1f}%')
print(f'Parameters: {total:,} (100% non-zero)')

# Pruning at different sparsity levels
print('\n📊 PRUNING AT DIFFERENT SPARSITY LEVELS')
print('=' * 60)

sparsity_levels = [0.5, 0.7, 0.9, 0.95, 0.99]
results = []

for sparsity in sparsity_levels:
    # Create fresh model and copy weights
    pruned_model = SimpleNet()
    pruned_model.load_state_dict(model.state_dict())
    
    # Apply pruning to all linear layers
    for name, module in pruned_model.named_modules():
        if isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=sparsity)
    
    # Accuracy before fine-tuning
    acc_no_ft = evaluate(pruned_model, test_loader)
    
    # Fine-tune for 3 epochs
    optimizer = torch.optim.Adam(pruned_model.parameters(), lr=0.0001)
    for epoch in range(3):
        pruned_model.train()
        for x, y in train_loader:
            optimizer.zero_grad()
            loss = criterion(pruned_model(x), y)
            loss.backward()
            # Zero out gradients of pruned weights
            for name, module in pruned_model.named_modules():
                if isinstance(module, nn.Linear):
                    module.weight.grad.data *= (module.weight.data != 0).float()
            optimizer.step()
    
    acc_with_ft = evaluate(pruned_model, test_loader)
    results.append((sparsity, acc_no_ft, acc_with_ft))
    print(f'Sparsity {sparsity*100:.0f}%: Before FT={acc_no_ft:.1f}%, After FT={acc_with_ft:.1f}%')

# Visualize results
fig, ax = plt.subplots(figsize=(10, 6))
sparsities = [r[0]*100 for r in results]
acc_no_ft = [r[1] for r in results]
acc_with_ft = [r[2] for r in results]

ax.plot(sparsities, acc_no_ft, 'o-', label='Without Fine-tuning', color='#ef4444', linewidth=2)
ax.plot(sparsities, acc_with_ft, 's-', label='With Fine-tuning', color='#22c55e', linewidth=2)
ax.axhline(y=acc_before, color='#3b82f6', linestyle='--', label=f'Original ({acc_before:.1f}%)')

ax.set_xlabel('Sparsity (%)', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('📈 Accuracy vs Sparsity', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\n💡 Key Insight: Fine-tuning recovers most accuracy loss from pruning!')

In [ ]:
print('🎯 KEY TAKEAWAYS')
print('=' * 60)
print('\n1. Many weights in trained networks are near-zero')
print('\n2. Magnitude pruning: Remove smallest absolute values')
print('\n3. Unstructured pruning: Higher sparsity, but harder to accelerate')
print('\n4. Structured pruning: Lower sparsity, but real speedups')
print('\n5. 2:4 Sparsity: Best of both - 2x speedup on NVIDIA GPUs')
print('\n6. Fine-tuning is crucial to recover accuracy')
print('\n' + '=' * 60)
print('\n📚 Next: Lottery Ticket Hypothesis and Advanced Pruning!')